<a href="https://colab.research.google.com/github/CristinaMarsh/Data/blob/main/2022_5_16_%E6%B0%94%E8%B1%A1%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/raspstephan/nwp-downscale.git

Cloning into 'nwp-downscale'...
remote: Enumerating objects: 2110, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 2110 (delta 53), reused 51 (delta 51), pack-reused 2018
Receiving objects: 100% (2110/2110), 156.72 MiB | 25.91 MiB/s, done.
Resolving deltas: 100% (1557/1557), done.


In [ ]:
!apt-get install libproj-dev proj-data proj-bin
!apt-get install libgeos-dev
!pip install cython
!pip install cartopy
!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely    # cartopy and shapely aren't friends (early 2020)
!pip install shapely --no-binary shapely

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np

import cartopy
import cartopy.crs as ccrs                   # for projections
import cartopy.feature as cfeature           # for features
import cartopy.io.shapereader as shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature.nightshade import Nightshade

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libproj-dev is already the newest version (4.9.3-2).
libproj-dev set to manually installed.
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  proj-bin
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 32.3 kB of archives.
After this operation, 110 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 32.3 kB in 0s (725 kB/s)
Selecting previously unselected package proj-bin.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../proj-bin_4.9.3-2_amd64.deb ...
Unpacking proj-bin (4.9.3-2) ...


In [ ]:
import xarray as xr
import pandas as pd
import urllib.request
import numpy as np
from glob import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os

In [ ]:
import cartopy.feature as cfeature
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

In [ ]:
url = 'url: https://cds.climate.copernicus.eu/api/v2'
key = 'key: 117001:448bd51e-d40e-4385-b677-f0c758c3ed76'

with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

with open('/root/.cdsapirc') as f:
    print(f.read())

url: https://cds.climate.copernicus.eu/api/v2
key: 117001:448bd51e-d40e-4385-b677-f0c758c3ed76


In [ ]:
! pip install cdsapi

  Created wheel for cdsapi: filename=cdsapi-0.5.1-py2.py3-none-any.whl size=11699 sha256=cf76043feedd0d090c9c06fd17cf9932f65bfa1de3044b9f5f4b8a275a570a76
  Stored in directory: /root/.cache/pip/wheels/1d/2e/3c/3746e0cd076320584d6f47b688da7b13a5d5d1a92606779ca4
Successfully built cdsapi


In [ ]:
import cdsapi
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': '2m_temperature',
        'year': '2020',
        'month': '12',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
    },
    'download.nc')

2022-05-15 20:58:37,564 INFO Welcome to the CDS
2022-05-15 20:58:37,565 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2022-05-15 20:58:37,697 INFO Request is queued
2022-05-15 20:58:38,740 INFO Request is running
2022-05-15 20:59:53,016 INFO Request is completed
2022-05-15 20:59:53,018 INFO Downloading https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data0/adaptor.mars.internal-1652648343.5862284-25611-14-1832ed36-65da-4e7e-8391-67361a348dc5.nc to download.nc (1.4G)
2022-05-15 21:00:54,572 INFO Download rate 23.9M/s


Result(content_length=1544913812,content_type=application/x-netcdf,location=https://download-0005-clone.copernicus-climate.eu/cache-compute-0005/cache/data0/adaptor.mars.internal-1652648343.5862284-25611-14-1832ed36-65da-4e7e-8391-67361a348dc5.nc)

In [ ]:
import xarray as xr
tigge_ctrl = xr.open_dataset('/content/download.nc')
#ds = xr.open_mfdataset(['test1.nc','test2.nc']) #for multiple files
tigge_ctrl

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 744)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2020-12-01 ... 2020-12-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-05-15 20:59:14 GMT by grib_to_netcdf-2.24.3: /opt/ecmw...

In [ ]:
tigge_ctrl.latitude.min()

<xarray.DataArray 'latitude' ()>
array(-90.)

In [ ]:
tigge_2dslice = tigge_ctrl.t2m.isel(lead_time=4, init_time=0)
p = tigge_2dslice.plot(   
    subplot_kws=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"),
    transform=ccrs.PlateCarree(),)
#p.axes.set_global()
p.axes.coastlines()